In [211]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut
import torch.nn as nn
import wandb
import torch
import torchvision 
import minmaxing as mm
random_seed = 42

In [212]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
#train = pd.read_csv('json_train.csv')
#train = train.drop("Unnamed: 0", axis = 1)
X_train,Y_train, X_test, Y_test = ut.preprocess_train(train, True, True, True)
mmvector = mm.get_vector_of_minmaxing(X_train)
#X_train = mm.minmax(X_train, mmvector)
#X_test = mm.minmax(X_test, mmvector)

c:\Users\Oleg\Desktop\dotakaggle\minmaxing.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[key] = {"min" : val.min(), "max" : val.max(), "std" : val.std(), "mean" : val.mean()}
c:\Users\Oleg\Desktop\dotakaggle\minmaxing.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[key] = {"min" : val.min(), "max" : val.max(), "std" : val.std(), "mean" : val.mean()}
c:\Users\Oleg\Desktop\dotakaggle\minmaxing.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

In [213]:
X_train

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
7676,675.0,22.0,7.0,1.0,0.0,74.0,0.0,1.0,0.0,10.0,...,12.997,0.0,0.0,2.0,0.0,0.667,0.0,0.0,0.0,0.0
1257,203.0,22.0,7.0,1.0,7.0,19.0,0.0,0.0,0.0,4.0,...,2.433,0.0,0.0,1.0,0.0,0.167,0.0,0.0,1.0,0.0
24896,1129.0,22.0,7.0,4.0,10.0,5.0,2.0,7.0,6.0,5.0,...,11.331,0.0,0.0,4.0,0.0,0.312,1.0,0.0,0.0,0.0
7056,1768.0,22.0,7.0,8.0,7.0,120.0,1.0,4.0,6.0,11.0,...,0.000,3.0,2.0,3.0,0.0,0.400,0.0,1.0,0.0,0.0
21579,1489.0,22.0,7.0,8.0,10.0,88.0,2.0,5.0,2.0,4.0,...,10.631,15.0,4.0,8.0,0.0,0.409,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23444,2434.0,22.0,0.0,16.0,0.0,91.0,2.0,10.0,20.0,0.0,...,1.733,0.0,0.0,2.0,0.0,0.591,0.0,0.0,9.0,2.0
9392,1040.0,22.0,7.0,4.0,12.0,12.0,8.0,4.0,7.0,6.0,...,7.598,0.0,0.0,0.0,0.0,0.158,0.0,0.0,0.0,0.0
1499,2102.0,22.0,7.0,10.0,23.0,41.0,4.0,12.0,3.0,2.0,...,0.000,0.0,0.0,6.0,1.0,0.500,0.0,0.0,11.0,14.0
27569,1148.0,22.0,7.0,4.0,4.0,62.0,1.0,7.0,7.0,0.0,...,0.000,3.0,1.0,4.0,0.0,0.188,0.0,0.0,2.0,3.0


In [214]:
train_dataset = ut.MyTensorDataset(torch.FloatTensor(X_train.values), torch.LongTensor(Y_train.values))
test_dataset = ut.MyTensorDataset(torch.FloatTensor(X_test.values), torch.LongTensor(Y_test.values))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=False,)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True,)

In [215]:
size = len(X_train.columns)
net = nn.Sequential(nn.Linear(size, 200), nn.Linear(200, 50), nn.Linear(50, 4), nn.Linear(4, 2),
                    nn.Softmax())
n_epoch = 20
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=0.1)

In [216]:
size

245

In [217]:
net[0].weight

Parameter containing:
tensor([[-0.0229, -0.0005,  0.0296,  ..., -0.0385, -0.0218, -0.0214],
        [ 0.0105, -0.0125,  0.0544,  ..., -0.0118, -0.0498, -0.0250],
        [-0.0243, -0.0167,  0.0501,  ...,  0.0117,  0.0141, -0.0277],
        ...,
        [ 0.0449,  0.0056, -0.0233,  ...,  0.0421,  0.0408,  0.0007],
        [-0.0457, -0.0466, -0.0265,  ..., -0.0033, -0.0243,  0.0194],
        [ 0.0075, -0.0476, -0.0100,  ..., -0.0493, -0.0144,  0.0330]],
       requires_grad=True)

In [218]:
loss_history = []

for i in range(n_epoch):
    for x_batch, y_batch in train_dataloader:
        #print(x_batch)
        y_batch = torch.transpose(y_batch, 0, 1)[0]
        #y_batch = y_batch[0]
        #print(y_batch)
        # Forward
        predictions = net(x_batch)
        #print(predictions)
        loss = criterion(predictions, y_batch)

        # Backward
        loss.backward()

        # Update weights
        optimizer.step()
        optimizer.zero_grad()

        loss_history.append(loss.item())
    print(f'{i+1}/{n_epoch} current loss: {loss}')

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


1/20 current loss: 1.31326162815094
2/20 current loss: 0.31326165795326233
3/20 current loss: 1.31326162815094
4/20 current loss: 0.31326165795326233
5/20 current loss: 0.31326165795326233
6/20 current loss: 0.31326165795326233
7/20 current loss: 1.31326162815094
8/20 current loss: 1.31326162815094
9/20 current loss: 0.31326165795326233
10/20 current loss: 0.31326165795326233
11/20 current loss: 0.31326165795326233
12/20 current loss: 0.31326165795326233
13/20 current loss: 0.31326165795326233
14/20 current loss: 1.31326162815094
15/20 current loss: 0.31326165795326233
16/20 current loss: 0.31326165795326233
17/20 current loss: 1.31326162815094
18/20 current loss: 1.31326162815094
19/20 current loss: 1.31326162815094
20/20 current loss: 0.31326165795326233


In [219]:
test_preds = ut.test_by_nn(net, test_dataloader, False)

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]], grad_fn=<SoftmaxBackward0>)
tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]], grad_fn=<SoftmaxBackward0>)
tensor([[1.0000e+00, 7.5375e-23],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00]], grad_fn=<SoftmaxBackward0>)
tensor([[1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 3.8768e-17],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00]], grad_fn=<SoftmaxBackward0>)
tensor([[1., 0.],
        [1., 0.],
        [1., 0.]

In [220]:
test_preds 

array([1., 1., 1., ..., 1., 1., 1.])

In [221]:
from sklearn.metrics import roc_auc_score
#print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, test_preds)}")

ROC-AUC score : 0.5


In [222]:
ut.writeNN(net, is_many_vals=False)

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([[1.0000e+00, 5.4296e-20],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00]], grad_fn=<SoftmaxBackward0>)
tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]], grad_fn=<SoftmaxBackward0>)
tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]], grad_fn=<SoftmaxBackward0>)
tensor([[1.0000e+00, 0.0000e+00],
        [1.0000e+00, 9.1882e-18],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00]], grad_fn=<SoftmaxBackward0>)
tensor([[1.0000e+00, 0.0000e+00],
        [1.0000e+0